# Домашнє завдання: Інтерактивні візуалізації з Plotly

## Опис завдання
У цьому домашньому завданні ви будете створювати інтерактивні візуалізації з допомогою бібліотеки Plotly. Ви дізнаєтесь різницю між Plotly Express (швидкі графіки) та Graph Objects (повний контроль), та створите інтерактивний дашборд.

**Опис колонок:**
- `datetime` - дата та час
- `season` - сезон (1=весна, 2=літо, 3=осінь, 4=зима)
- `holiday` - чи є день святковим (0=ні, 1=так)
- `workingday` - чи є день робочим (0=ні, 1=так)
- `weather` - погодні умови (1=ясно, 2=туман, 3=легкий дощ, 4=сильний дощ)
- `temp` - температура в градусах Цельсія
- `atemp` - відчувається як температура
- `humidity` - вологість (%)
- `windspeed` - швидкість вітру
- `casual` - кількість випадкових користувачів
- `registered` - кількість зареєстрованих користувачів
- `count` - загальна кількість орендованих велосипедів

## Підготовка даних


In [37]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Завантаження даних
df = pd.read_csv('../data/yulu_rental.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

# Для plotly краще не встановлювати datetime як індекс
df['month'] = df['datetime'].dt.month
df['hour'] = df['datetime'].dt.hour
df['weekday'] = df['datetime'].dt.day_name()

# Додаємо назви сезонів
season_map = {1: 'Весна', 2: 'Літо', 3: 'Осінь', 4: 'Зима'}
df['season_name'] = df['season'].map(season_map)

## Завдання 1: Базовий інтерактивний лінійний графік (Plotly Express)

**Завдання:**
Створіть інтерактивний лінійний графік динаміки оренди за часом (рівень деталізації - як в даних) з можливістю zoom та hover.

Дайте відповіді на питання.
**Питання для інтерпретації:**
1. Яка перевага інтерактивного графіка над статичним?
2. Чому на графіку є "пробіли" - ділянки, де одна пряма лінія зʼєднує два "суцільних" блоки з даними? Як би ви це могли дослідити на статичному графіку?


In [38]:

df['month_year'] = df['datetime'].dt.to_period('M').astype(str)

monthly_avg = df.groupby('month_year')['count'].mean().reset_index()

fig = px.line(
    monthly_avg,
    x='month_year',
    y='count',
    markers=True,
    title='📈 Середня кількість оренд велосипедів помісячно',
    labels={'month_year': 'Місяць-Рік', 'count': 'Середня кількість оренд'}
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

Дайте відповіді на питання.
**Питання для інтерпретації:**
1. Яка перевага інтерактивного графіка над статичним?

    Інтерактивність: можна навести курсор і побачити точне значення.
    Масштабування: можна збільшити конкретну ділянку часу (zoom).

2. Чому на графіку є "пробіли" - ділянки, де одна пряма лінія зʼєднує два "суцільних" блоки з даними? Як би ви це могли дослідити на статичному графіку?
    Можливо бо відсутні данні в певні проміжки 

## Завдання 2: Scatter plot з додатковими даними (Plotly Express)

**Завдання:**
Створіть scatter plot кількості орендованих велосипедів випадковими користувачами vs кількості орендованих велосипедів зареєстрованими користувачами. Розмір точок встановіть за сумарною кількістю велосипедів, які були взяті в оренду, а колір - за сезоном. В hover_data - додайте деталі, які допоможуть вам в подальшому аналізі.

Дослідіть графік. Зверніть увагу, що ви можете вмикати і вимикати окремі сезони, якщо будете клікати на колір сезону в легенді графіку.

**Дайте відповідь на питання.**
- Як ви проінтерпретуєте роздвоєність цього графіку (дві явні лінії)? Що це означає?
- Які висновки для компанії, які дає велосипеди в оренду, ви можете зробити з цього графіку? 3 основних висновки.

In [39]:
fig = px.scatter(
    df,
    x='casual',
    y='registered',
    size='count',
    color='workingday',
    hover_data=['datetime', 'workingday', 'holiday', 'weather', 'temp'],
    title='Кількість оренд: випадкові vs зареєстровані користувачі',
    size_max=15
)

fig.update_traces(marker=dict(sizemode='area', line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(legend_title='Сезон')
fig.show()

**Дайте відповідь на питання.**
- Як ви проінтерпретуєте роздвоєність цього графіку (дві явні лінії)? Що це означає?

     Компанія має дві ключові цільові аудиторії:
         Registered (регулярні, щоденні поїздки)
         Casual користувачі (вихідні, дозвілля, туризм)


- Які висновки для компанії, які дає велосипеди в оренду, ви можете зробити з цього графіку? 3 основних висновки.

    На графіку видно чітку сегментацію користувачів, що можна використати для оптимізації маркетингу і пошуку шляхів розвитку Casual користувачів



## Завдання 3: Порівняння Plotly Express vs Graph Objects

**Завдання:**
Створіть лінійний графік помісячної динаміки оренди велосипедів двома способами - з Plotly Express та з Graph Objects.

**Дайте відповіді на питання.**
1. Як ви розумієте основну різницю між цими двома підходами?
2. Коли краще використовувати Plotly Express?
3. Коли потрібен Graph Objects?


In [40]:
fig = go.Figure()

#Plotly Express
fig_express = px.line(
    monthly_avg,
    x='month_year',
    y='count',
    title='📈 Середня кількість оренд помісячно (Plotly Express)',
    markers=True,
    labels={'month_year': 'Місяць', 'count': 'Середня кількість оренд'}
)
fig_express.update_layout(xaxis_tickangle=-45)
fig_express.show() 


#Graph Objects

fig_go = go.Figure()
fig_go.add_trace(go.Scatter(
    x=monthly_avg['month_year'],
    y=monthly_avg['count'],
    mode='lines+markers',
    name='Оренди',
    line=dict(color='royalblue'),
    marker=dict(size=6)
))
fig_go.update_layout(
    title='📊 Середня кількість оренд помісячно (Graph Objects)',
    xaxis_title='Місяць',
    yaxis_title='Середня кількість оренд',
    xaxis_tickangle=-45
)
fig_go.show()




**Дайте відповіді на питання.**
1. Як ви розумієте основну різницю між цими двома підходами?

    Plotly Express має простіший синтаксис але Plotly Graph гнучкий і дає більше контролю над графіком 

2. Коли краще використовувати Plotly Express?

    Коли треба швидко побудувати графік. Коли потрібна типова візуалізація (лінії, bar, scatter). Якщо не треба складного оформлення або багатьох графіків разом.
    
3. Коли потрібен Graph Objects?

    Коли потрібна повна кастомізація графіка (кольори, стилі, шрифти, лейбли, layout) та Для інтерактивних дешбордів або презентацій

## Завдання 4: Дашборд з make_subplots (Graph Objects)

**Завдання:**
Створіть дашборд з 4 різними графіками в одній фігурі:
- Bar chart - середні значення загальної кількості оренд велосипедів за сезонами
- Pie chart - відсоткове співвідношення погодних умов в даних
- Line chart - середнє значення загальної кількості оренд велосипедів за годинами протягом доби
- Scatter plot - кореляція температури vs вологість

Додайте заголовок на дашборд.

**Дайте відповідь на питання**
- На ваш погляд, яка перевага об'єднання графіків в один дашборд?

In [41]:

#  Bar chart: середні оренди за сезонами
bar_data = df.groupby('season')['count'].mean().reset_index()

#  Pie chart: частка погодних умов
weather_data = df['weather'].value_counts().reset_index()
weather_data.columns = ['weather', 'count']

#  Line chart: середні оренди за годинами
hourly_avg = df.groupby('hour')['count'].mean().reset_index()

# Scatter plot: температура vs вологість
scatter_x = df['temp']
scatter_y = df['humidity']

# Subplot (2x2)
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Середня кількість оренд за сезонами",
        "Розподіл погодних умов",
        "Середня кількість оренд за годинами",
        "Температура vs Вологість"
    ),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "scatter"}, {"type": "scatter"}]]
)

# Bar chart
fig.add_trace(go.Bar(x=bar_data['season'], y=bar_data['count'], name='Season Avg'), row=1, col=1)

# Pie chart
fig.add_trace(go.Pie(labels=weather_data['weather'], values=weather_data['count'], name='Weather Share'), row=1, col=2)

# Line chart
fig.add_trace(go.Scatter(x=hourly_avg['hour'], y=hourly_avg['count'], mode='lines+markers', name='Hourly Avg'), row=2, col=1)

# Scatter plot
fig.add_trace(go.Scatter(x=scatter_x, y=scatter_y, mode='markers', name='Temp vs Humidity'), row=2, col=2)

# Оформлення
fig.update_layout(
    height=800,
    width=1000,
    title_text="🚲 Дашборд: Аналіз оренди велосипедів",
    showlegend=False
)

fig.show()

Яка перевага об'єднання графіків в один дашборд?

Переваги: 

Цілісне сприйняття даних — ти бачиш одразу кілька аспектів (сезонність, погоду, години, кореляції) в одному вікні.
    
Зручна презентація — зручно показувати команді, менеджменту чи в звітах.

## Завдання 5: 3D візуалізація

**Завдання:**
Створіть 3D scatter plot для аналізу взаємозв'язку температури, швидкості вітру та загальної кількості орендованих велосипедів. Колір встановіть за сезоном, а розмір - за загальною кількість оренд також.

Дайте відповіді на питання.
**Питання для інтерпретації:**
1. Яку додаткову інформацію, на ваш погляд, дає 3D візуалізація?
2. Чи видно кластери в 3D просторі?
3. Чи ви можете зробити висновки з цієї візуалізації, чи вам було простіше побудувати кілька 2D?



In [47]:
fig = px.scatter_3d(
    df,
    x='temp',
    y='windspeed',
    z='count',
    color='season',
    size='count',
    title='3D аналіз: Температура, Вітер та Кількість оренд',
    labels={
        'temp': 'Температура',
        'windspeed': 'Швидкість вітру',
        'count': 'Кількість оренд',
        'season': 'Сезон'
    },
    opacity=1
)

fig.update_layout(margin=dict(l=0, r=0, b=0, t=40))
fig.show()

1. Яку додаткову інформацію дає 3D візуалізація?

      1.1 Дає змогу одночасно оцінити вплив трьох змінних: temp, windspeed та count.

      1.2 Візуально легко побачити, де найвища концентрація оренд за поєднанням погодних факторів.


2. Чи видно кластери в 3D просторі?
Можна побачити кілька кастерів: 
      
      Область високої температури і слабкого вітру → більше оренд.

      Область низької температури або сильного вітру → менше оренд.


## Завдання 6: Експорт та збереження інтерактивних графіків

**Завдання:**
Збережіть побудований раніше дашборд в формат HTML. Також змініть вручну щось на дашборді (зум, виділення частини графіку) і збережіть його як статичне зображення через іконку фотоапарату у формат PNG. Завантажте файли з дашбордом у HTML та PNG (або посилання на них на github) разом з посиланням на цей ноутбук при здачі ДЗ.
